In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import copy
# from graphviz import Digraph
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict
from ClfTemplates import get_all_template_path, sample_template, sample_template_path
from ClfUse import showClfResult, get_samples, labeledDataFromJson
from Ner_model import get_X_Y, lstm_crf, predict, preprocess
from Ner_data_make import create_fake_label, fake2real
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, SpatialDropout1D,concatenate, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras.utils import to_categorical
import keras
from collections import Iterable 
import pandas as pd
import numpy as np
import random
import re
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd
import random

def gen_new(ori_str):
    return ''.join([str(random.randint(0,9)) if i.isdigit() else i for i in list(ori_str)])

def create_fake_label(DF, fake_times):
    DF2 = DF.copy()
    for i in range(fake_times):
        col = DF2.apply(lambda x: gen_new(x.token) if x.lable!='other' else x.token, axis=1)
        DF2['new_tokens_{}'.format(i)] = col
    return DF2

def generate_sent_id(sent_id_list, origin_sent_id):
    dic = {}
    for k in origin_sent_id:
        tmp = 'send_id_' + str(sent_id_list[-1])
        sent_id_list.pop()
        dic.setdefault(k, tmp)
    return sent_id_list, dic

def fake2real(df, fake_times):
    '''label 写错了，将错就错'''
    lis = []
    sent_id_list = list(range(len(df.sent_id.unique()) * fake_times))
    origin_sent_id = df.sent_id.unique()
    for i in range(fake_times):
        tmp_df = df[['new_tokens_{}'.format(i), 'lable', 'sent_id']]
        tmp_df.columns = ['token', 'lable', 'sent_id']
        sent_id_list, dic = generate_sent_id(sent_id_list, origin_sent_id)
        tmp_df.sent_id = tmp_df.sent_id.map(dic)
        lis.append(tmp_df)
    return pd.concat(lis)

def fake2real_v2(df, fake_times):
    lis = []
    sent_id_list = list(range(len(df.sent_id.unique()) * fake_times))
    origin_sent_id = df.sent_id.unique()
    for i in range(fake_times):
        tmp_df = df[['new_tokens_{}'.format(i), 'sent_id', 'cls']]
        tmp_df.columns = ['token', 'sent_id', 'cls']
        sent_id_list, dic = generate_sent_id(sent_id_list, origin_sent_id)
        tmp_df.sent_id = tmp_df.sent_id.map(dic)
        lis.append(tmp_df)
    return pd.concat(lis)

### 1. sms 文件

In [3]:
import pickle
import os
from IPython.display import display
import itertools
# /home/qibo/sms_nlp
res_root = '/home/qibo/sms_nlp/annotation_results/round_2_results_bank/'
# res_root = '/home/qibo/work_notebooks/sms_nlp/annotation_results/round_2_results_loan/'
ALL = []
for idx,fn in enumerate(os.listdir(res_root)):
    print(fn)
    despatcher = fn.split('.')[0]
    print(despatcher)
    with open(os.path.join(res_root, fn), 'rb') as fin:
        tasks = pickle.load(fin)

    for key, group in itertools.groupby(tasks, key=lambda x:x.output[0]):
#         print(list(group)[-2].output)
#         print('##############################')
#         print(list(group)[-1].output)
        regex_id = key
        to_display = None
        sms_type = None
        fields = []
        field_types = []
        for t in list(group):
            output = t.output
            val = t.value
            regex_id = output[0]
            group_id = output[1]
            gened_regex = output[2]
            box_markups = output[3]
            if group_id == 0:
                sms_type = val
                if sms_type:
                    print(regex_id, gened_regex)
                    print('短信类型: ', sms_type)
                to_display = box_markups[0][0]
                text = box_markups[0][0].text
#                 print('text :', text)
                spans = box_markups[0][0].spans
                for s in spans:
                    fields.append(text[s.start : s.stop])
            else:
                field_types.append(val)
        print(field_types)
        if None not in field_types:
            tem = {}
            for i , j in zip(fields, field_types):
                tem[i] = j
            new_text = 'Reg_id_' + str(regex_id) + ' ' + text
#             print(cluster_id)
            ALL.append([text, fields ,field_types, tem, sms_type])
        print('------------------------------------------------------')

indusb.pkl
indusb
3764 ^Dear Customer Txn on IndusInd Bank Credit Card no .* for INR .* on .* at .* is .* call 18602677777 for query\.Click on http\:\/\/bit\.ly\/2B8nSNl to update your Aadhaar Number\, ignore if already done\.$
短信类型:  交易流水＿转账
['银行卡号＿自己', '金额＿转出', '日期＿交易时间', '机构＿交易平台', 'other']
------------------------------------------------------
3765 ^Your IndusInd Bank A\/C .* has been debited for .* towards your Debit Card purchase\.The available Balance is .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '金额＿余额']
------------------------------------------------------
3766 ^Your a\/c no\. .* is credited by .* on .* by a\/c linked to mobile .* \(IMPS Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
3767 ^Your IndusInd Bank A\/C .* has been debited for .* towards an ATM Cash Withdrawal Transaction\. Available Balance is .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '金额＿余额']
------------------------------------------

['账户账号＿自己', '金额＿转出', '日期＿交易时间', 'other', None]
------------------------------------------------------
9351 ^Dear Customer\, Your a\/c no\. .* is debited for .* on .* and a\/c .* credited \(IMPS Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
9352 ^Dear Customer\, Your a\/c no\. .* is credited by .* on .* by a\/c linked to mobile .* \(IMPS Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
9353 ^You have initiated a txn of INR .* at .* on ICICI Bank Card .* OTP is .* DONT SHARE WITH ANYONE\. BANK NEVER CALLS TO VERIFY OTP\.$
短信类型:  交易流水＿转账
['金额＿转出', '机构＿交易平台', '银行卡号＿自己', 'other']
------------------------------------------------------
9354 ^Dear Customer\, your Account .* has been credited with INR .* on .* Info\: .* The Available Balance is INR .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', 'other', '金额＿余额']
----------

------------------------------------------------------
['账户账号＿自己', None, None, None]
------------------------------------------------------
5147 ^Dear Customer\,Your SB Account No .* is credited by LPG subsidy Rs .* on .* Avbl Bal .* DISCOUNT CODE FNP15UBI FOR FERNSNPETALS$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', '金额＿余额']
------------------------------------------------------
[None, None, None, None, None, None, None, None]
------------------------------------------------------
5150 ^Dear customer\,perhaps you have missed payment of your installment of .* in your A\/c .* with Union Bank\.Kindly ignore if already paid$
短信类型:  sms_other
['other', 'other']
------------------------------------------------------
5151 ^Dear Customer\,Please make Payment of Rs\. .* due in your A\/c .* with Union Bank at Branch .* immediately\.Kindly Ignore .*$
短信类型:  信用卡＿逾期警告
['金额＿应还金额', '账户账号＿自己', 'other', 'other']
------------------------------------------------------
5153 ^Bank is replacing your exi

9947 ^Hello\! You have initiated to add .* to your account .* Please keep the txn id .* for future reference$
短信类型:  交易流水＿转账
['金额＿转入', '账户账号＿自己', 'other']
------------------------------------------------------
[None, None, None, None]
------------------------------------------------------
9949 ^Your a\/c no\. .* is debited for .* on .* and credited to .* \(UPI Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
9953 ^SUCCESSFUL transfer of Rs .* to .* using Airtel Payments Bank transfer by \- .* ID\: .* Charges \- Max .*$
短信类型:  交易流水＿转账
['金额＿转出', '机构＿交易平台', '账户账号＿他人', 'other', 'other']
------------------------------------------------------
9957 ^Hello\! You have added .* in your Airtel Payments Bank A\/C\. Avl Bal .* Txn ID\. .*$
短信类型:  交易流水＿转账
['金额＿转入', '金额＿余额', 'other']
------------------------------------------------------
9960 ^You have paid .* using your Online Card at .* IND\. Avl Bal .* Txn ID .* Co

[None, None, None, None, None, None]
------------------------------------------------------
[None, None, None, None, None, None]
------------------------------------------------------
[None, None, None, None, None, None]
------------------------------------------------------
1806 ^Your a\/c no\. .* is debited for .* on .* and a\/c .* credited \(IMPS Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
1807 ^Cash deposit of INR .* on .* made in A\/c .* at PMC Bank\. Clr Bal\:INR .* Eff Avail Bal\:INR .*$
短信类型:  交易流水＿转账
['金额＿转入', '日期＿交易时间', '账户账号＿自己', '金额＿信用额度', '金额＿余额']
------------------------------------------------------
1808 ^Your a\/c no\. .* is credited for .* on .* by a\/c linked to mobile .* \(IMPS Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
1811 ^Your a\/c no\. .* is debited for .* on .* and credited to VPA .* \

...

5174 ^A charge of Rs\. .* has been initiated on your RBL Credit Card ending with .* at .* on .* at .* IST\. Your available limit is Rs\. .*$
短信类型:  交易流水＿转账
['金额＿转出', '银行卡号＿自己', '机构＿交易平台', '日期＿交易时间', 'other', '金额＿信用额度']
------------------------------------------------------
5176 ^INR .* debited from a\/c .* on .* by use of Card no ending .* at .* Avail Bal .* For assistance call 18001238040$
短信类型:  交易流水＿转账
['金额＿转出', '账户账号＿自己', '日期＿还款日期', '银行卡号＿自己', '机构＿交易平台', '金额＿余额']
------------------------------------------------------
5177 ^Dear .* Transfer of .* to .* has been successful with REF .*$
短信类型:  交易流水＿转账
['other', '金额＿转出', '机构＿交易平台', 'other']
------------------------------------------------------
5180 ^Your a\/c no\. .* is debited for .* on .* and credited to a\/c no\. .* \(UPI Ref no .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
5182 ^Dear Cardmember\, payment of .* has been received towards your RBL Bank Cr

3224 ^Rs .* debited from a\/c .* on .* to VPA .* Ref No .* Not you\? Call on .* to report$
短信类型:  交易流水＿转账
['金额＿转出', '账户账号＿自己', '日期＿交易时间', 'other', 'other', 'other']
------------------------------------------------------
3226 ^Rs\. .* credited to a\/c .* on .* by a\/c linked to VPA .* \(UPI Ref No .*$
短信类型:  交易流水＿转账
['金额＿转入', '账户账号＿自己', '日期＿交易时间', 'other', 'other']
------------------------------------------------------
3229 ^Rs\. .* debited from a\/c .* on .* to a\/c .* \(UPI Ref No\. .* Not you\? Call on .* to report$
短信类型:  交易流水＿转账
['金额＿转出', '账户账号＿自己', '日期＿交易时间', '账户账号＿他人', 'other', 'other']
------------------------------------------------------
3231 ^.* was withdrawn using your HDFC Bank Card ending .* on .* at .* Avl bal\: .*$
短信类型:  交易流水＿转账
['金额＿转出', '银行卡号＿自己', '日期＿交易时间', '机构＿交易平台', '金额＿余额']
------------------------------------------------------
3232 ^.* was spent on ur HDFCBank CREDIT Card ending .* on .* at .* bal \- .* curr o\/s \- .*$
短信类型:  交易流水＿转账
['金额＿转出', '银行卡号＿自己', '日期＿交易时

8475 ^.* EMI of .* is due on .* for .* order .* You will have to maintain sufficient balance in your account for auto\-debit\. .*$
短信类型:  信用卡＿还款提醒
['other', '金额＿应还金额', '日期＿还款日期', 'other', 'other', 'other']
------------------------------------------------------
8476 ^.* EMI of .* is due on .* for .* order .* You will have to maintain sufficient balance in your account for auto\-debit\.$
短信类型:  信用卡＿还款提醒
['other', '金额＿应还金额', '日期＿还款日期', 'other', 'other']
------------------------------------------------------
8477 ^Hi\, your ZestMoney EMI application for .* of .* has been cancelled\. Any payment made will be instantly refunded and depending on your bank will reflect in your account within 7\-10 days\. Help us improve our service by sharing your feedback \- .*$
短信类型:  贷前申请＿审核拒绝
['机构＿交易平台', '金额＿放款金额', 'other']
------------------------------------------------------
8479 ^.* you are the best\. The payment of .* for your ZestMoney .* was successful\. Login to your ZestMoney account to track your

['账户账号＿自己', None, None, None, None, None]
------------------------------------------------------
[None, None, None, None, None]
------------------------------------------------------
4026 ^Your A\/c No .* has been debited by Rs\. .* on .* via .* A\/c No .* Bal is Rs\. .* CR and AVL Bal is Rs\. .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', 'other', 'other', '金额＿余额', '金额＿信用额度']
------------------------------------------------------
4028 ^Your A\/c\. .* is credited by Rs\. .* on .* by A\/c linked to mobile .* \( IMPS Ref no\. .* \)\.$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', '账户账号＿他人', 'other']
------------------------------------------------------
4030 ^Your A\/c No .* has been credited by .* on .* by IMPS .* from .* A\/c Bal is Rs\. .* CR and AVL Bal is Rs\. .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转入', '日期＿交易时间', 'other', '机构＿交易平台', '金额＿余额', '金额＿信用额度']
------------------------------------------------------
4032 ^Your A\/C .* Balance is\: .* CR\;Available Balance is\: .*$
短信类型:  交易

[None, None, None, None, None]
------------------------------------------------------
[None, None, None, None, None, None, None]
------------------------------------------------------
5801 ^Your a\/c no .* is debited for .* on .* and a\/c .* of .* is credited \(IMPS Ref no .* \)\.$
短信类型:  交易流水＿转账
['账户账号＿自己', '金额＿转出', '日期＿交易时间', '账户账号＿他人', 'other', 'other']
------------------------------------------------------
5803 ^Dear Customer\, Your request for IMPS payment of Rs\. .* for the beneficiary IFSC .* a\/c .* has been .* From CorpBank$
短信类型:  交易流水＿转账
['金额＿转出', 'other', '账户账号＿自己', 'other']
------------------------------------------------------
5805 ^Your A\/c .* in .* credited INR .* on .* by CASH\. Avl Bal is .*$
短信类型:  交易流水＿转账
['账户账号＿自己', '机构＿交易平台', '金额＿转入', '日期＿交易时间', '金额＿余额']
------------------------------------------------------
[None, None, None, None, None]
------------------------------------------------------
5814 ^Your A\/c .* in .* debited INR .* on .* by CASH\. Avl Bal is .*$


In [4]:
DF = pd.DataFrame()
for tt in ALL:
    DF_tmp = pd.DataFrame()
    length = len(tt[0].split(' '))
    DF_tmp['token'] = tt[0].split(' ')
    DF_tmp['lable'] = DF_tmp.token.apply(lambda x: tt[3][x] if x in tt[3] else np.nan)
    DF_tmp['sent_id'] = [tt[0].split(' ')[i] if i==0 else np.nan for i in range(len(DF_tmp))]
    DF_tmp['cls'] = [tt[4]]*length
    DF_tmp.sent_id=DF_tmp.sent_id.fillna(method='ffill')
    DF = DF.append(DF_tmp)
    DF = DF[~DF.cls.isnull()]
DF = DF.fillna(value='other')

DF['sent_id'] = ['sent_' if i == 0 else np.nan for i in DF.index]
DF['idx_num'] = range(DF.shape[0])
DF['sent_id'] = DF.apply(lambda x: str(x.sent_id) + str(x.idx_num), axis=1)
DF['sent_id'] = DF.sent_id.apply(lambda x: x if x.startswith('sent') else np.nan)
DF.sent_id = DF.sent_id.fillna(method='ffill')
DF = DF.drop(['idx_num'], axis=1)
DF = DF[DF.cls != False]
DF.shape

(29300, 4)

### 唯一测试集；

In [5]:
Test_DF = pd.read_csv('gsmTst100.csv', header=None).iloc[:,1:]
Test_DF.columns = ['token', 'lable', 'sent_id', 'cls']

sms_te = Test_DF.groupby('sent_id').apply(lambda x: ' '.join(x.token.tolist()).lower())
cls_te = Test_DF.groupby('sent_id').apply(lambda x: x.cls.tolist()[0])
df3_te = pd.DataFrame(columns=['sms', 'cls'])
df3_te.sms = sms_te
df3_te.cls = cls_te

df3_te.shape


(100, 2)

In [6]:
DF = DF[~DF.sent_id.isin(Test_DF.sent_id)]
DF.shape

(26883, 4)

In [7]:
## 造假n倍
dupli = 10
DF = DF[DF.cls != False]
df = create_fake_label(DF, dupli)
df2 = fake2real_v2(df, dupli)

In [7]:
# 过采样欠采样；2bDone

In [8]:
def get_cos_similarity(sms, templates):
    '''计算一条新sms 与 每个 template 相似度;'''
    def cos_sim(a, b):
        return dot(a, b) / (norm(a) * norm(b))
    return [cos_sim(i, sms) for i in templates]


def tfIdfVector(corpus):
    '''
    corpus is a list of sentences:
    ['This is an example', 'hello world', ...]
    '''
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    x = vectorizer.fit_transform(corpus)
    tfidf = transformer.fit_transform(x)
    return tfidf.toarray()

# def newAndTemplate(template_df, new_sms, thresh=None):
#     '''
#     new_sms: test samples
#     '''
#     corpus, labels = template_df.sms.tolist(), template_df.label.tolist()
#     num_sms = len(new_sms)
#     for single in new_sms:
#         corpus.append(single.lower())
#     all_tfidf = tfIdfVector(corpus)
#     template_tfidf = all_tfidf[:-num_sms]
#     instances_tfidf = all_tfidf[-num_sms:]
#     adj_mat = cosine_similarity(all_tfidf)
#     print('adj mat shape:{}'.format(adj_mat.shape))
#     if thresh:
#         print('thresh:{}'.format(thresh))
#         adj_mat-=np.identity(len(adj_mat))
#         G2 = nx.from_numpy_matrix((adj_mat > thresh)*1)
#     else:
#         G2 = nx.from_numpy_matrix(adj_mat)
#     print(G2.number_of_edges())
#     for i in range(len(G2.nodes())):
#         G2.node[i]['vec'] =  all_tfidf[i]
#         G2.node[i]['text'] = corpus[i]
#         temp_label = labels[i] if i < len(labels) else 0
#         G2.node[i]['label'] = temp_label
#     return G2

def newAndTemplate2(template_df, new_sms, thresh=None):
    '''
    new_sms: test samples, df type;
    '''
    corpus, labels = template_df.sms.tolist(), template_df.cls.tolist()
    new_labels = new_sms.cls.tolist()
    new_sms = new_sms.sms.tolist()
    num_sms = len(new_sms)
    for single in new_sms:
        corpus.append(single.lower())
    for single in new_labels:
        labels.append(single)
    all_tfidf = tfIdfVector(corpus)
    adj_mat = cosine_similarity(all_tfidf)
    print('adj mat shape:{}'.format(adj_mat.shape))
    if thresh:
        print('thresh:{}'.format(thresh))
        adj_mat-=np.identity(len(adj_mat))
        G2 = nx.from_numpy_matrix((adj_mat > thresh)*1)
    else:
        G2 = nx.from_numpy_matrix(adj_mat)
    print(G2.number_of_edges())
    for i in range(len(G2.nodes())):
        G2.node[i]['vec'] =  all_tfidf[i]
        G2.node[i]['text'] = corpus[i]
        G2.node[i]['label'] = labels[i]
    return G2

In [10]:
df2 = DF

In [11]:
sms = df2.groupby('sent_id').apply(lambda x: ' '.join(x.token.tolist()).lower())
cls = df2.groupby('sent_id').apply(lambda x: x.cls[0])
df3 = pd.DataFrame(columns=['sms', 'cls'])
df3.sms = sms
df3.cls = cls

In [12]:
g5 = newAndTemplate2(template_df = df3, new_sms = df3_te)
test_nodes = list(g5.nodes)[-100:]

adj mat shape:(1195, 1195)
701206


In [ ]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())


In [13]:
def sampleNeighs4Node_i(G, i, num):
    '''
    G: graph
    i: node_idx.
    '''
    z = [nbrdict for n, nbrdict in G.adjacency()]
    sorted_weight = sorted(z[i].items(), key=lambda x: x[1]['weight'],reverse=True)
    return sorted_weight[1:1+num]

def getBatchAdjMat1(G, batchNodeIdx,num):
    ''' 
    原文中为了保证同一个batch shape 一致性;
    同时为了 A@emb 同时纳入(结构特征 与 点特征);
    把列映射为batch level的unique nodes;
    这样让A得以成为adj 矩阵；
    我这里如果同样处理方式，则有可能将adj[i][i] 带进来；无法避免这种可能性；
    可以解决； mask 元素相乘；
    batchNodeIdx: each batch like:[1,2,3,4,5,...]
    G:graph
    '''
    sample = [i[0] for nodeid in batchNodeIdx
                for i in sampleNeighs4Node_i(G, nodeid, num)]
    cols = np.unique(sample)
    mask = np.zeros((len(batchNodeIdx), len(cols)))
    print('mask shape:{}'.format(mask.shape))
    for idx, i in enumerate(batchNodeIdx):
        for jdx, j in enumerate(cols):
            mask[idx][jdx] = G.adj[i][j]['weight']
    normalized = np.array(mask)/np.array(mask).sum(axis=-1)[:,np.newaxis]
    return mask, normalized

def getBatchAdjMat2(G, batchNodeIdx, num):
    ''' 
    可以保证我的batch shape 永远一致，我不受采样约束，
    因为是按weight rank 采样;
    batchNodeIdx: each batch like:[1,2,3,4,5,...]
    G: graph;
    '''
    sample = [[i[1]['weight'] for i in sampleNeighs4Node_i(G, nodeid, num)]
              for nodeid in batchNodeIdx]
    normalized = np.array(sample)/np.array(sample).sum(axis=-1)[:,np.newaxis]
    return np.array(sample), normalized


def mask_adj(G, batchNodeIdx, num):
    # IO: G, batchNodeIdx,num
    sample2 = [[i[0] for i in sampleNeighs4Node_i(G, nodeid, num)]
              for nodeid in batchNodeIdx]
    sample = [i[0] for nodeid in batchNodeIdx
                for i in sampleNeighs4Node_i(G, nodeid, num)]
    unique_nodes = np.unique(sample)
    node2idx ={i:j for j,i in enumerate(unique_nodes)}
    mask = np.zeros((len(batchNodeIdx), len(unique_nodes)))
    # IO: sample2, idx2node, mask
    rows_ = [ii for ii in range(len(sample2)) for j in range(len(sample2[ii]))]
    cols_ = [node2idx[j] for i in sample2 for j in i]
    mask[rows_,cols_] = 1
    return mask, unique_nodes


In [14]:
"""
Set of modules for aggregating embeddings of neighbors.
"""
class MeanAggregator_QB(nn.Module):
    """
    # 要改， 采样的时候，按照 weight batch 采样 top_N； 
    # mask = [N, Unique_batch_nodes] entrys are weight again.
    # 然后归一化mask@emb_mat = 【N, F】
    # 然后【N,F】@[F,num_cls] = [N,num_cls]
    """
    def __init__(self, G, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.

        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator_QB, self).__init__()
        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        self.G = G
        
    def forward(self, nodes, num_sample=10):
        """
        nodes --- list of nodes in a batch, [bs]
        num_sample --- number of neighbors to sample. 
        """
        # 三种mask_adj;
        # 第一种是做unique col map, 会部分引入sim(i,i)=1, 
        # 没有关系原文也会隐式引入;
        ############################################
#         _, norm_adj = getBatchAdjMat2(self.G, nodes, num_sample)
#         mask = torch.from_numpy(norm_adj).type(torch.float)
        
        # 第二种是类比原文，adj@emb = [bs, F];
        ############################################
        mask, unique_nodes_list = mask_adj(self.G, nodes, num_sample)
        mask = torch.from_numpy(mask).type(torch.float)
        
        # 第三种是不做unique映射；能做 adj@emb 吗？
        # 不能，因为adj 的特定含义（点，点关系）无法表达。
        # unique 映射 是必须的。
        ##########################################
#         _, norm_adj = getBatchAdjMat1(self.G, nodes, num_sample)
#         mask = torch.from_numpy(norm_adj).type(torch.float)
        
        if self.cuda:
            mask = mask.cuda()
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
            
        to_feats = mask.mm(embed_matrix) #[bs, unique]@[unique, F]
        return to_feats # [bs, F]



class Encoder_QB(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim,
            embed_dim, aggregator, num_sample,
            gcn=False, cuda=False): 
        super(Encoder_QB, self).__init__()
        self.features = features
        self.aggregator = aggregator
        self.num_sample = num_sample
        self.feat_dim = feature_dim
        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes; [bs]
        operations: 已知self_feat: [bs, F]
                    再agg_feat: [bs, F]
                    然后concat([agg_feat, self_feat]): [bs, 2F]
                    然后W:[F2, 2F] 
                    relu(W @[bs,2F].T) : [F2, bs] 
        """
        # [bs, F]
        neigh_feats = self.aggregator.forward(nodes, self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        # [hid, bs] = relu([hid, 2f]@[2f, bs]) 
        combined = F.relu(self.weight.mm(combined.t()))
        return combined


class SupervisedGraphSage_QB(nn.Module):
    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage_QB, self).__init__()
        self.enc = enc
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)
        self.loss = nn.NLLLoss(reduce=False)
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, nodes, softmax=False):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        if softmax:
            return self.softmax(scores)
        return scores.t()

    def loss_softmax_mask(self, nodes, labels, mask=None):
        '''
        nodes shape: [N];
        labels shape: [N];
        mask shape: [N];
        '''
        if mask is None:
            mask = torch.ones(len(labels))
        logits = self.forward(nodes)
        A_soft = self.logsoftmax(logits)
        output = self.loss(A_soft, labels.squeeze())
        loss = output*mask.type(torch.float)
        loss = sum(loss)/sum(mask)
        return loss
    
def result(G, output, label_idx, idx2label):
    score, pred = output.max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[g4.node[i]['label'] for i in label_idx]
    return df

In [15]:
g4 = g5

In [16]:
np.random.seed(1)
random.seed(1)
num_nodes = len(g4.nodes)
node_F = len(g4.node[0]['vec'])
feat_data = [g4.node[i]['vec'] for i in g4.nodes]
labels = np.array([g4.node[i]['label'] for i in g4.nodes])
idx2label = {i:j for i, j in enumerate(np.unique(labels))}
label2idx = {j:i for i,j in idx2label.items()}
labels = np.array([label2idx[i] for i in labels])
num_cls = len(np.unique(labels))
# feat_data, labels, adj_lists, _ = load_cora()
features = nn.Embedding(num_nodes, node_F)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)

agg1 = MeanAggregator_QB(g4, features, cuda=True)
enc1 = Encoder_QB(features, node_F, 128, agg1, num_sample=5, gcn=False, cuda=False)
agg2 = MeanAggregator_QB(g4, lambda nodes : enc1(nodes).t(), cuda=False)
enc2 = Encoder_QB(lambda nodes : enc1(nodes).t(), 
               enc1.embed_dim, node_F, agg2, num_sample=5, gcn=False, cuda=False)


In [17]:
graphsage = SupervisedGraphSage_QB(num_cls, enc2)
#graphsage.cuda()
#rand_indices = np.random.permutation(len(node_list))
nodes = list(g4.nodes)
val = nodes[:50]
train = nodes[50:-len(test_nodes)]
optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr = 0.1)
times = []
Loss = []

for batch in range(1):
    batch_nodes2 = train[:40]
    batch_idx = copy.copy(train[:40])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss_softmax_mask(batch_nodes2,
            Variable(torch.tensor(labels[np.array(batch_idx)])))
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time - start_time)
    val_output = graphsage.forward(val, softmax=True)
    print(batch, loss.data)
    Loss.append(loss.data)

0 tensor(2.8286)


In [19]:
torch.cuda.is_available()

False

In [18]:
val_output = graphsage.forward(val)
result(g4, val_output, val, idx2label)

,sms,score,pred,label
0,dear customer txn on indusind bank credit card...,0.204012,交易流水＿转账,交易流水＿转账
1,your ecs request dated 10-sep-18 towards satya...,0.205185,交易流水＿转账,账号异常＿余额不足
2,rs 400 was withdrawn at an atm from a/c xx5819...,0.200824,交易流水＿转账,交易流水＿转账
3,your fund transfer for rs. 30.65 on 07-jul-18 ...,0.260191,交易流水＿转账,账号异常＿余额不足
4,"payment received for an amount of rs.5,000.00 ...",0.230175,交易流水＿转账,交易流水＿转账
5,payment of rs.6600.00 received for card ******...,0.218801,交易流水＿转账,交易流水＿转账
6,"dear citi customer, your online payment of rs....",0.219899,交易流水＿转账,交易流水＿转账
7,"dear citi card member, congratulations you hav...",0.214929,交易流水＿转账,交易流水＿转账
8,"payment of rs. 5,000.00 received for card no 4...",0.213352,交易流水＿转账,交易流水＿转账
9,the balance in your a/c xx2708 is rs.711.93. n...,0.166306,交易流水＿转账,交易流水＿余额


In [ ]:
te_output = graphsage.forward(test_nodes)
result(g4, te_output, test_nodes, idx2label)

In [ ]:
# valid 
Res = pd.DataFrame(columns=['true', 'pred'])

true, pred = get_prediction(model, X_word_val, X_char_val, y_val)

Res['true'] = true
Res['pred'] = pred

entitys = Res.true.unique()
records = []
for i in entitys:
    tmp = evaluate(Res, i)
    records.append(tmp)
record = pd.DataFrame.from_records(records)
record.columns =['cls','精确率','召回率', 'F1', 'support']
record = record.set_index('cls')
record = record.sort_index()
record

In [ ]:
grouped_te = Test_DF.groupby('sent_id').apply(lambda x: [(w.lower(), t) for w,t in zip(x.token.tolist(), x.lable.tolist())])
sentences_te = [s for s in grouped_te]

X_word_te = [[word2idx.get(w[0], word2idx['UNK']) for w in s] for s in sentences_te]
X_word_te = pad_sequences(maxlen=max_len, sequences=X_word_te, value=word2idx['PAD'],padding='post',truncating='post')

X_char_te = get_X_char(sentences_te)
y_te = [[tag2idx[w[1]] for w in s] for s in sentences_te]
y_te = pad_sequences(maxlen=max_len, sequences=y_te, value=tag2idx['other'],padding='post',truncating='post')


In [ ]:
# test
Res = pd.DataFrame(columns=['true', 'pred'])
true, pred = get_prediction(model, X_word_te, X_char_te, y_te)
Res['true'] = true
Res['pred'] = pred

entitys = Res.true.unique()
records = []
for i in entitys:
    tmp = evaluate(Res, i)
    records.append(tmp)
record = pd.DataFrame.from_records(records)
record.columns =['cls','精确率','召回率', 'F1','support']
record = record.set_index('cls')
record = record.sort_index()
record

In [ ]:
grouped_te = Test_DF.groupby('sent_id').apply(lambda x: [(w.lower(), t) for w,t in zip(x.token.tolist(), x.lable.tolist())])
sentences_te = [s for s in grouped_te]

X_word_te = [[word2idx.get(w[0], word2idx['UNK']) for w in s] for s in sentences_te]
X_word_te = pad_sequences(maxlen=max_len, sequences=X_word_te, value=word2idx['PAD'],padding='post',truncating='post')

X_char_te = get_X_char(sentences_te)
y_te = [[tag2idx[w[1]] for w in s] for s in sentences_te]
y_te = pad_sequences(maxlen=max_len, sequences=y_te, value=tag2idx['other'],padding='post',truncating='post')


In [ ]:
gsm_path = 'gsm_templates_df.csv'
gsm_templates_df = pd.read_csv(gsm_path)
gsm_templates_df = gsm_templates_df[~gsm_templates_df.label.isnull()]
print(gsm_templates_df.shape)

In [ ]:
test_100example = gsm_templates_df.sample(n=100)
gsm_templates_df = gsm_templates_df[~gsm_templates_df.index.isin(test_100example.index)]
print(gsm_templates_df.shape)

In [24]:
def result(G, output, label_idx, idx2label):
    score, pred = output.max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[g4.node[i]['label'] for i in label_idx]
    return df

In [ ]:
result(g4, val_output, val, idx2label)

In [ ]:
idx2label[np.argmax(graphsage.forward([3]).squeeze(0).data.numpy())]

In [ ]:
def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}
 
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
 
    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'
 
    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot